In [1]:
"""
Full disclosure: This version works best on a Jupyter Notebook server.
Only difference with VSCode is the inclusion of an output widget to display HTML properly on the second Code Cell.
If run on VSCode with Jupyter extension, it might have trouble rendering the output Widget as it should.
There seems to be documented issues and difference between the two as shown here: https://github.com/microsoft/vscode-jupyter/issues/9503
"""
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from link_builder import get_rec_link
from recmegpt_module import RecMeGPT

<h3 >Welcome to RecMeGPT! </h3>
Discover your next obsession with RecMeGPT! Unleash a world of entertainment tailored just for you! From mind-bending albums to immersive games, cinematic gems, gripping TV shows, and literary masterpieces. Explore, experience, and elevate your entertainment journey with our curated suggestions. Your next favorite thing awaits!<br>
You can enter anything from any media you like and even love!<br>
From a theme to a song or band or movie and anything in between! We will do the best to get you going!

In [2]:
# The textbox for the user to input what they need and the button to call the GPT Model and show the results
user_input = widgets.Text( value='', placeholder='Type something you like!', description='Your input:')
gpt_button = widgets.Button(description='Get your recs!', tooltip='Click me!')
output_area = widgets.Output()

def get_recommendations(a):
    """
    Calls the class RecMeGPT to try to get the recommendation based on the textbox.
    If sucessfull it will the display them, if not it will tell the user it couldn't be done and to try again.
    Args:
    - a (widgets.Button): The input by the user representing what they like and want to get recommendations based off.    
    Returns:
    - None
    """
         
    def display_recs(recs,header,index):
        """
        Inner function to display the recommendations in HTML tags
         Args:
        - recs (list): List of dict with the recommendations for a specific media
        - header (str): Value of the HTML header
        - index (int):  Numeric identifier of the media, used to create hyperlinks for each recommendation  
        Returns:
        - None
        """
        displayed_text=f"<h2>{header}</h2>"
        for rec in recs:
            displayed_text += f'<h3>{rec["title"]}</h3>{rec["description"]} - {get_rec_link(rec["title"],index)} <br>'
        with output_area:
            #clear_output(wait=True)
            display(HTML(displayed_text))

    try:
        # Clear output, disables interactive objects so the user cannot send a new request if one is already running        
        user_input.disabled = True
        gpt_button.disabled = True
                
        # Check if the input was empty, if so, it will warn the user to fix it
        if not user_input.value:
            raise Exception("<h2>Oops!</h2>You forgot to input something! Please try again!")
        with output_area:
            clear_output(wait=True)
            display(HTML("<h3>Looking for recommendations...</h3>"))
        
        # Create an object of the class in charge of interacting with GPT
        recme_gpt = RecMeGPT()
        # Set the response to 'data', 'data' will be False if there was an issue on the object function
        data = recme_gpt.request_recommendation(user_input.value.strip(),1)
        
        #testing with local json
        #import json
        #with open('test_from_web.json','r') as file:
        #    data = json.load(file)


        # If there was an issue with the 'data', warns the user
        if not data:
            raise Exception("<h2>Oops!</h2>Something went wrong! Please try again with another input!")
        
        # If the data was OK, it will show the data of the recommendations
        if data["status"] == "OK":            
            html_headers=["Music Albums", "Videogames", "Movies", "TV Shows", "Books"]
            with output_area:
                display(HTML(f"<h1>Your recommendations!</h1>{data['header']}" ))
            for i, recs in enumerate(data["recs"].values()):
                display_recs(recs, html_headers[i], i)
        #if the data set by the GPT Model, but the model couldn't find anything, it warn the user
        else:
            with output_area:
                clear_output(wait=True)
                display(HTML(f"<h2>Oops!</h2>I could not find anything, sorry! Please try again!" ))
        
    except Exception as e:
        error = str(e)
        if error[:4] != '<h2>':
            error = "<h2>Oops!</h2>Something went wrong! Please try again!"
        with output_area:
            clear_output(wait=True)
            display(HTML(error))
    finally:
        # Re-enable the interactive objects, so the user can try again
        user_input.disabled = False
        gpt_button.disabled = False


# Set the function to run when the button is click and shows the interactive objects for the first time
#gpt_button.on_click(get_recommendations)

gpt_button.on_click(get_recommendations)
display(user_input)
display(gpt_button, output_area)


Text(value='', description='Your input:', placeholder='Type something you like!')

Button(description='Get your recs!', style=ButtonStyle(), tooltip='Click me!')

Output()